In [17]:
UNIV_COLUMNS = ['대학코드', '국가', '대학명', '파견구분', '협정형태', 'VISIT_NO', "BERT_SUM", "RNN_SUM", 'href']
ABSTRACT_REVIEWS_COLUMNS = ['대학코드', '제목', '학과', '과정', '년도', 'BERT_SCORE', 'RNN_SCORE', 'href']
SPECIFIC_COLUMNS = ["대학코드", "대학명", "국가", "파견구분", "협정형태", "학과", "제목", "과정", "년도", "college_link", "review_link",  "gen_info","env_info","food_info","study_info","office_info" "facil_info","mhct_info","help_info","etc_info", "BERT_SCORE", "RNN_SCORE"]

In [3]:
import os
from os import fdopen, remove
import glob
from tempfile import mkstemp
import shutil
from shutil import move, copymode

import pandas as pd
import numpy as np

In [4]:
# define path for sentiment labeled dataset
# abstract: 짧은 후기 제목만 있는 텍스트 데이터셋
# specific: 문단으로 구성된 텍스트 데이터셋

abstract_sentiment = "./data_sentiment/abstract"
specific_sentiment = "./data_sentiment/specific"

In [5]:
# fetch all yonsei exchange review text datasets for each foreign universities
abstract_yonsei_reviews = glob.glob(f"{abstract_sentiment}/*.csv")
abstract_yonsei_reviews[:5]

['./data_sentiment/abstract/NL000006_review_abstract_sentiment.csv',
 './data_sentiment/abstract/FI000014_review_abstract_sentiment.csv',
 './data_sentiment/abstract/US000183_review_abstract_sentiment.csv',
 './data_sentiment/abstract/US000109_review_abstract_sentiment.csv',
 './data_sentiment/abstract/CA000002_review_abstract_sentiment.csv']

In [6]:
# look at sample dataset
sample_file = abstract_yonsei_reviews[70]
file_name = sample_file.split("/")[-1]
file_name_without_ext = file_name.split(".")[0]
print(file_name_without_ext)
univ_code = file_name_without_ext.split("_")[0]
print(univ_code)
df_abstract = pd.read_csv(sample_file, encoding="utf-8")
df_abstract.sample(5)

AU000016_review_abstract_sentiment
AU000016


,Unnamed: 0,No,제목,학과,과정,년도,href,BERT_SCORE,RNN_SCORE
3,3,44,시드니에서의 한 학기,응용통계학과,학부,2013,/partner/expReport.asp?id=6898&page=1&bgbn=R,1,0.719899
21,21,26,내 인생의 못 잊을 1년,영어영문학과,학부,2006-2007,/partner/expReport.asp?id=2723&page=3&bgbn=R,0,0.742134
15,15,32,잊지 못할 UTS 교환학생 한 학기,사회학과,학부,2008,/partner/expReport.asp?id=3234&page=2&bgbn=R,0,0.834257
0,0,47,"Warm, Welcoming, and Stimulating UTS","UIC, TAD (IID & CDM)",학부,2018,/partner/expReport.asp?id=14839&page=1&bgbn=R,0,0.516349
13,13,34,내 인생 최고의 Holiday,주거환경학과,학부,2008,/partner/expReport.asp?id=3499&page=2&bgbn=R,1,0.932632


In [7]:
df_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review["대학코드"] = univ_code
_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review["대학코드"] = univ_code
df_abstract_review = df_abstract_review[["대학코드","제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
df_abstract_review.head()

,대학코드,제목,학과,과정,년도,BERT_SCORE,RNN_SCORE,href
0,AU000016,"Warm, Welcoming, and Stimulating UTS","UIC, TAD (IID & CDM)",학부,2018,0,0.516349,/partner/expReport.asp?id=14839&page=1&bgbn=R
1,AU000016,다시 돌아가고 싶은 UTS,의공학부,학부,2017,1,0.913062,/partner/expReport.asp?id=14400&page=1&bgbn=R
2,AU000016,행복했던 시드니에서 한학기,기계공학과,학부,2016,1,0.772298,/partner/expReport.asp?id=13591&page=1&bgbn=R
3,AU000016,시드니에서의 한 학기,응용통계학과,학부,2013,1,0.719899,/partner/expReport.asp?id=6898&page=1&bgbn=R
4,AU000016,시드니에서의 1년,경영학과,학부,2012-2013,0,0.719899,/partner/expReport.asp?id=6422&page=1&bgbn=R


In [15]:
df_abstract_review.columns.to_list()

['대학코드', '제목', '학과', '과정', '년도', 'BERT_SCORE', 'RNN_SCORE', 'href']

In [13]:
def make_abstract_review_df(file_path):
    df_abstract = pd.read_csv(file_path, encoding="utf-8")
    df_abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    df_abstract_review["대학코드"] = univ_code
    _abstract_review = df_abstract[["제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    df_abstract_review["대학코드"] = univ_code
    df_abstract_review = df_abstract_review[["대학코드","제목", "학과", "과정", "년도", "BERT_SCORE", "RNN_SCORE", "href"]]
    return df_abstract_review

In [9]:
df_univ = pd.read_csv("./data/univ_db_full.csv", encoding="utf-8")
df_univ.sample(5)

,level_0,index,No,대학명,국가,파견구분,협정형태,href,visitation
62,62,15,16,University of Regina,CANADA,NaN,ISEP,/partner/expReport.asp?ucode=CA000021&bgbn=A,Exists
398,398,5,6,London Business School,UNITED KINGDOM,교환학생,NaN,/partner/expReport.asp?ucode=GB000001&bgbn=A,Exists
685,685,249,250,University of Oregon,UNITED STATES,교환학생,교환대학,/partner/expReport.asp?ucode=US000143&bgbn=A,Exists
479,479,43,44,CSU Channel Islands,UNITED STATES,교환학생,교환대학,/partner/expReport.asp?ucode=US000023&bgbn=A,None
232,232,4,5,Doshisha University,JAPAN,NaN,교환대학,/partner/expReport.asp?ucode=JP000005&bgbn=A,Exists


In [10]:
series_query = df_univ["href"].str.split("=", expand=True)[1]
series_univ_code = series_query.str.split("&", expand=True)[0]
df_univ["대학코드"] = series_univ_code

In [11]:
df_univ.head()

,level_0,index,No,대학명,국가,파견구분,협정형태,href,visitation,대학코드
0,0,0,1,Kabul University,AFGHANISTAN,교환학생,교환대학,/partner/expReport.asp?ucode=AF000001&bgbn=A,None,AF000001
1,1,0,1,Universidad Blas Pascal,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000001&bgbn=A,None,AR000001
2,2,1,2,Universidad Catolica de Cordoba,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000002&bgbn=A,None,AR000002
3,3,2,3,Universidad de Palermo,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000003&bgbn=A,None,AR000003
4,4,3,4,Universidad del Salvador,ARGENTINA,교환학생,ISEP,/partner/expReport.asp?ucode=AR000004&bgbn=A,Exists,AR000004


In [12]:
df = df_univ[["대학코드", "국가", "대학명", "파견구분", "협정형태", "visitation", "href"]]
df.head()

,대학코드,국가,대학명,파견구분,협정형태,visitation,href
0,AF000001,AFGHANISTAN,Kabul University,교환학생,교환대학,None,/partner/expReport.asp?ucode=AF000001&bgbn=A
1,AR000001,ARGENTINA,Universidad Blas Pascal,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000001&bgbn=A
2,AR000002,ARGENTINA,Universidad Catolica de Cordoba,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000002&bgbn=A
3,AR000003,ARGENTINA,Universidad de Palermo,교환학생,ISEP,None,/partner/expReport.asp?ucode=AR000003&bgbn=A
4,AR000004,ARGENTINA,Universidad del Salvador,교환학생,ISEP,Exists,/partner/expReport.asp?ucode=AR000004&bgbn=A


In [16]:
df.columns.to_list()

['대학코드', '국가', '대학명', '파견구분', '협정형태', 'visitation', 'href']

In [14]:
def make_univ_list_df():
    df_univ = pd.read_csv("./data/univ_db_full.csv", encoding="utf-8")
    series_query = df_univ["href"].str.split("=", expand=True)[1]
    series_univ_code = series_query.str.split("&", expand=True)[0]
    df_univ["대학코드"] = series_univ_code
    df = df_univ[["대학코드", "국가", "대학명", "파견구분", "협정형태", "visitation", "href"]]
    return df